In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
url = "https://athletics.wheaton.edu/sports/baseball/schedule/2023"
response = requests.get(url)
if response.status_code == 200:
    soup = BeautifulSoup(response.text, 'html.parser')
    box_score_urls = []
    for a in soup.find_all('a', href=True):
        if 'boxscore' in a['href']:
            box_score_urls.append(a['href'])

else:
    print("Failed to retrieve the webpage. Status code:", response.status_code)

chopped_box_score_urls = box_score_urls[::2]
base_url = 'https://athletics.wheaton.edu'
full_box_score_list = [base_url + url for url in chopped_box_score_urls]
full_box_score_list

dataframes = []

for url in full_box_score_list:
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    plays = []
    play_section = soup.find_all('table', class_='sidearm-table play-by-play')
    for play in play_section:
        for item in play.find_all('tr'):
            plays.append(item.get_text(strip=True))
    
    midpoint = len(plays)//2
    plays = plays[:midpoint]
    
    for play in plays:
        if play[-1].isdigit():
            for i in range(len(plays)):
                if plays[i][-1].isdigit():
                    plays[i] = plays[i][:-1]

    if 'Play Description' not in plays[-1]:
        plays.append('Play Description')
    
    df = pd.DataFrame(plays, columns=['Play'])
    
    outs = 0
    outs_before_play = [0]
    for play in plays:
        if outs == 3:
            outs = 0
            outs_before_play.append(outs)
        elif 'grounded out' in play:
            outs += 1
            outs_before_play.append(outs)
        elif 'flied out' in play:
            outs += 1
            if 'out at' in play:
                outs += 1
            outs_before_play.append(outs)
        elif 'fouled out' in play:
            outs += 1
            if 'out at' in play:
                outs += 1
            outs_before_play.append(outs)
        elif 'double play' in play:
            outs += 2
            outs_before_play.append(outs)
        elif 'popped up' in play:
            outs += 1
            if 'out at' in play:
                outs += 1
            outs_before_play.append(outs)
        elif 'lined out' in play:
            outs += 1
            if 'out at' in play:
                outs += 1
            outs_before_play.append(outs)
        elif 'struck out' in play:
            outs += 1
            outs_before_play.append(outs)
        elif "fielder's choice" and "out at" in play:
            outs += 1
            outs_before_play.append(outs)
        elif 'caught stealing' in play:
            outs += 1
            outs_before_play.append(outs)
        else:
            outs_before_play.append(outs)
    outs_before_play_fixed = outs_before_play[:-1]
    df['Outs Before Play'] = outs_before_play_fixed
    
    count = -1
    ro1 = 0
    ro2 = 0
    ro3 = 0
    ro1_bp = [0]
    ro2_bp = [0]
    ro3_bp = [0]
    for play in plays:
        count += 1
        if outs_before_play[count] == 3:
            ro1 = 0
            ro2 = 0
            ro3 = 0
            
        if 'walked' in play:
            ro1 = 1
            if 'advanced to second' in play:
                ro2 = 1
            if 'advanced to third' in play:
                ro3 = 1
                
        if 'hit by pitch' in play:
            ro1 = 1
            if 'advanced to second' in play:
                ro2 = 1
            if 'advanced to third' in play:
                ro3 = 1
    
        if 'singled' in play:
            ro1 = 1
            if 'scored' in play:
                ro3 = 0
                ro2 = 0
                if 'advanced to third;' in play:
                    ro3 = 1
                if 'advanced to second;' in play:
                    ro2 = 1
            if 'advanced to third' in play:
                if ', advanced to third' in play:
                    if 'advanced to second;' not in play:
                        ro1 = 0
                        ro3 = 1
                    else:
                        ro3 = 1
                else:
                    ro3 = 1
                    ro2 = 0
            if 'advanced to second' in play:
                if ', advanced to second' in play:
                    ro1 = 0
                    ro2 = 1
                else:
                    ro2 = 1
            if 'out at' in play:
                if 'at second' in play:
                    ro2 = 0
                if 'at third' in play:
                    ro3 = 0
                if 'at home' in play:
                    if 'advanced to third;' not in play:
                        ro3 = 0
                        
        if 'doubled' in play:
            ro1 = 0
            ro2 = 1
            if 'scored' in play:
                ro3 = 0
                if 'advanced to third;' in play:
                    ro3 = 1
            if 'advanced to third' in play:
                if ', advanced to third' in play:
                    ro2 = 0
                    ro3 = 1
                else:
                    ro3 = 1
            if 'out at' in play:
                if 'at third' in play:
                    ro3 = 0
                if 'at home' in play:
                    if 'advanced to third;' not in play:
                        ro3 = 0
    
        if 'tripled' in play:
            ro1 = 0
            ro2 = 0
            ro3 = 1
            if ', scored' in play:
                ro3 = 0
    
        if 'homered' in play:
            ro1 = 0
            ro2 = 0
            ro3 = 0
    
        if 'reached' in play:
            ro1 = 1
            if 'scored' in play:
                ro3 = 0
                ro2 = 0
                if 'advanced to third;' in play:
                    ro3 = 1
                if 'advanced to second;' in play:
                    ro2 = 1
            if 'advanced to third' in play:
                ro3 = 1
                if 'advanced to second' not in play:
                    ro2 = 0
                    if ', advanced to third' in play:
                        ro1 = 0
            if 'advanced to second' in play:
                if ', advanced to second' in play:
                    ro1 = 0
                    ro2 = 1
                else:
                    ro2 = 1
            if 'out at' in play:
                if 'at second' in play:
                    ro2 = 0
                if 'at third' in play:
                    ro3 = 0
                if 'at home' in play:
                    if 'advanced to third;' not in play:
                        ro3 = 0
    
        if 'flied out' in play:
            if 'scored' in play:
                ro3 = 0
                if 'scored;' in play:
                    ro2 = 0
            if 'advanced to third' in play:
                ro2 = 0
                ro3 = 1
            if 'advanced to second' in play:
                ro1 = 0
                ro2 = 1
            if 'out at' in play:
                if 'at second' in play:
                    ro2 = 0
                if 'at third' in play:
                    ro3 = 0
                if 'at home' in play:
                    if 'advanced to third;' not in play:
                        ro3 = 0

        if 'fouled out' in play:
            if 'scored' in play:
                ro3 = 0
                if 'scored;' in play:
                    ro2 = 0
            if 'advanced to third' in play:
                ro2 = 0
                ro3 = 1
            if 'advanced to second' in play:
                ro1 = 0
                ro2 = 1
            if 'out at' in play:
                if 'at second' in play:
                    ro2 = 0
                if 'at third' in play:
                    ro3 = 0
                if 'at home' in play:
                    if 'advanced to third;' not in play:
                        ro3 = 0
    
        if 'grounded out' in play:
            ro1 = 0
            if 'scored' in play:
                ro3 = 0
                if 'scored;' in play:
                    ro2 = 0
            if 'advanced to third' in play:
                ro2 = 0
                ro3 = 1
            if 'advanced to second' in play:
                ro1 = 0
                ro2 = 1
    
        if 'lined out' in play:
            if 'scored' in play:
                ro3 = 0
                if 'scored;' in play:
                    ro2 = 0
            if 'advanced to third' in play:
                ro2 = 0
                ro3 = 1
            if 'advanced to second' in play:
                ro1 = 0
                ro2 = 1
            if 'out at' in play:
                if 'at second' in play:
                    ro2 = 0
                if 'at third' in play:
                    ro3 = 0
                if 'at home' in play:
                    if 'advanced to third;' not in play:
                        ro3 = 0
    
        if 'popped up' in play:
            if 'scored' in play:
                ro3 = 0
                if 'scored;' in play:
                    ro2 = 0
            if 'advanced to third' in play:
                ro2 = 0
                ro3 = 1
            if 'advanced to second' in play:
                ro1 = 0
                ro2 = 1
            if 'out at' in play:
                if 'at second' in play:
                    ro2 = 0
                if 'at third' in play:
                    ro3 = 0
                if 'at home' in play:
                    if 'advanced to third;' not in play:
                        ro3 = 0
    
        if 'double play' in play:
            ro2 = 0
            ro1 = 0
            if 'scored' in play:
                ro3 = 0
            if 'advanced to third' in play:
                ro3 = 1
            if 'advanced to second' in play:
                ro2 = 1
                
        if 'wild pitch' in play:
            if 'scored' in play:
                ro3 = 0
            if 'advanced to third' in play:
                ro2 = 0
                ro3 = 1
            if 'advanced to second' in play:
                ro1 = 0
                ro2 = 1
    
        if 'passed ball' in play:
            if 'scored' in play:
                ro3 = 0
            if 'advanced to third' in play:
                ro2 = 0
                ro3 = 1
            if 'advanced to second' in play:
                ro1 = 0
                ro2 = 1
    
        if 'balk' in play:
            if 'scored' in play:
                ro3 = 0
            if 'advanced to third' in play:
                ro2 = 0
                ro3 = 1
            if 'advanced to second' in play:
                ro1 = 0
                ro2 = 1
    
        if 'stole' in play:
            if 'stole home' in play:
                ro3 = 0
            if 'stole third' in play:
                ro2 = 0
                ro3 = 1
            if 'stole second' in play:
                ro1 = 0
                ro2 = 1
    
        if 'caught stealing' in play:
            if 'out at second' in play:
                ro1 = 0
            if 'out at third' in play:
                ro2 = 0
            if 'out at home' in play:
                ro3 = 0
            if 'advanced to' in play:
                if 'advanced to second' in play:
                    ro2 = 1
                if 'advanced to third' in play:
                    ro3 = 1
                if 'scored' in play:
                    ro3 = 0
            
        ro1_bp.append(ro1)
        ro2_bp.append(ro2)
        ro3_bp.append(ro3)
    
    ro1_bp_fixed = ro1_bp[:-1]
    ro2_bp_fixed = ro2_bp[:-1]
    ro3_bp_fixed = ro3_bp[:-1]
    df['Runners On 1st Before'] = ro1_bp_fixed
    df['Runners On 2nd Before'] = ro2_bp_fixed
    df['Runners On 3rd Before'] = ro3_bp_fixed
    
    visitor = 0
    home = 1
    vscore = 0
    hscore = 0
    vscore_bp = [0]
    hscore_bp = [0]
    for play in plays:
        if 'Play Description' in play:
            if visitor == 0:
                visitor = 1
                home = 0
            elif visitor == 1:
                visitor = 0
                home = 1
        if visitor == 1:
            if 'homered' in play:
                vscore += 1
            vscore += play.count('scored')
            vscore_bp.append(vscore)
            hscore_bp.append(hscore)
        elif home == 1:
            if 'homered' in play:
                hscore += 1
            hscore += play.count('scored')
            vscore_bp.append(vscore)
            hscore_bp.append(hscore)
    
    vscore_bp_fixed = vscore_bp[:-1]
    hscore_bp_fixed = hscore_bp[:-1]
    df['Visitor Score Before Play'] = vscore_bp_fixed
    df['Home Score Before Play'] = hscore_bp_fixed
    
    rroi = 0
    rroi_list = [0]
    count = -1
    reoi = 0
    for play in plays:
        rroi = 0
        reoi = 0
        count += 1
        count2 = 0
        for play2 in plays[count:]:
            if 'Play Description' in play2:
                reoi = vscore_bp[count+count2]+hscore_bp[count+count2]
                break
            else: count2 += 1
        rroi = reoi - (vscore_bp[count] + hscore_bp[count])
        rroi_list.append(rroi)
    
    rroi_list_fixed = rroi_list[1:]
    df['Runs Scored Rest of Inning'] = rroi_list_fixed

    dataframes.append(df)


In [2]:
merged_df = pd.concat(dataframes, ignore_index=True)

In [3]:
filtered_df = merged_df[~merged_df['Play'].str.contains('Play Description', na=False)]
filtered_df = filtered_df[~filtered_df['Play'].str.contains('to p for', na=False)]
filtered_df = filtered_df[~filtered_df['Play'].str.contains('to c for', na=False)]
filtered_df = filtered_df[~filtered_df['Play'].str.contains('to 1b for', na=False)]
filtered_df = filtered_df[~filtered_df['Play'].str.contains('to 2b for', na=False)]
filtered_df = filtered_df[~filtered_df['Play'].str.contains('to 3b for', na=False)]
filtered_df = filtered_df[~filtered_df['Play'].str.contains('to ss for', na=False)]
filtered_df = filtered_df[~filtered_df['Play'].str.contains('to lf for', na=False)]
filtered_df = filtered_df[~filtered_df['Play'].str.contains('to cf for', na=False)]
filtered_df = filtered_df[~filtered_df['Play'].str.contains('to rf for', na=False)]
filtered_df = filtered_df[~filtered_df['Play'].str.contains('to dh for', na=False)]
filtered_df = filtered_df[~filtered_df['Play'].str.contains('pinch hit', na=False)]
filtered_df = filtered_df[~filtered_df['Play'].str.contains('pinch ran', na=False)]

##Removing misceallneous non-plays
filtered_df = filtered_df.drop(index = 321) 
filtered_df = filtered_df.drop(index = 323)
filtered_df = filtered_df.drop(index = 324)
filtered_df = filtered_df.drop(index = 325)
filtered_df = filtered_df.drop(index = 326)
filtered_df = filtered_df.drop(index = 328)
filtered_df = filtered_df.drop(index = 329)
filtered_df = filtered_df.drop(index = 339)
filtered_df = filtered_df.drop(index = 340)
filtered_df = filtered_df.drop(index = 351)
filtered_df = filtered_df.drop(index = 771)
filtered_df = filtered_df.drop(index = 932)
filtered_df = filtered_df.drop(index = 1156)
filtered_df = filtered_df.drop(index = 1157)
filtered_df = filtered_df.drop(index = 1158)
filtered_df = filtered_df.drop(index = 1175)
filtered_df = filtered_df.drop(index = 1177)
filtered_df = filtered_df.drop(index = 1178)
filtered_df = filtered_df.drop(index = 1185)
filtered_df = filtered_df.drop(index = 1186)
filtered_df = filtered_df.drop(index = 1188)
filtered_df = filtered_df.drop(index = 1747)
filtered_df = filtered_df.drop(index = 1949)
filtered_df = filtered_df.drop(index = 2184)
filtered_df = filtered_df.drop(index = 2244)
filtered_df = filtered_df.drop(index = 2314)
filtered_df = filtered_df.drop(index = 2315)
filtered_df = filtered_df.drop(index = 2316)
filtered_df = filtered_df.drop(index = 2318)
filtered_df = filtered_df.drop(index = 2391)
filtered_df = filtered_df.drop(index = 2392)
filtered_df = filtered_df.drop(index = 2970)
filtered_df = filtered_df.drop(index = 2971)
filtered_df = filtered_df.drop(index = 2973)
filtered_df = filtered_df.drop(index = 3181)
filtered_df = filtered_df.drop(index = 3430)
filtered_df = filtered_df.drop(index = 3545)
filtered_df = filtered_df.drop(index = 3546)
filtered_df = filtered_df.drop(index = 3547)
filtered_df = filtered_df.drop(index = 3548)
filtered_df = filtered_df.drop(index = 3677)
filtered_df = filtered_df.drop(index = 3678)
filtered_df = filtered_df.drop(index = 3800)
filtered_df = filtered_df.drop(index = 3839)
filtered_df = filtered_df.drop(index = 3887)
filtered_df = filtered_df.drop(index = 3888)
filtered_df = filtered_df.drop(index = 4479)
filtered_df = filtered_df.drop(index = 4481)

In [7]:
df_0o_01_02_03 = filtered_df[
    (filtered_df['Outs Before Play'] == 0) &
    (filtered_df['Runners On 1st Before'] == 0) &
    (filtered_df['Runners On 2nd Before'] == 0) &
    (filtered_df['Runners On 3rd Before'] == 0)]

pd.set_option('display.max_rows', None)

In [11]:
df_1o_01_02_03 = filtered_df[
    (filtered_df['Outs Before Play'] == 1) &
    (filtered_df['Runners On 1st Before'] == 0) &
    (filtered_df['Runners On 2nd Before'] == 0) &
    (filtered_df['Runners On 3rd Before'] == 0)]

In [15]:
df_2o_01_02_03 = filtered_df[
    (filtered_df['Outs Before Play'] == 2) &
    (filtered_df['Runners On 1st Before'] == 0) &
    (filtered_df['Runners On 2nd Before'] == 0) &
    (filtered_df['Runners On 3rd Before'] == 0)]

In [19]:
df_0o_11_02_03 = filtered_df[
    (filtered_df['Outs Before Play'] == 0) &
    (filtered_df['Runners On 1st Before'] == 1) &
    (filtered_df['Runners On 2nd Before'] == 0) &
    (filtered_df['Runners On 3rd Before'] == 0)]

In [21]:
df_1o_11_02_03 = filtered_df[
    (filtered_df['Outs Before Play'] == 1) &
    (filtered_df['Runners On 1st Before'] == 1) &
    (filtered_df['Runners On 2nd Before'] == 0) &
    (filtered_df['Runners On 3rd Before'] == 0)]

In [23]:
df_2o_11_02_03 = filtered_df[
    (filtered_df['Outs Before Play'] == 2) &
    (filtered_df['Runners On 1st Before'] == 1) &
    (filtered_df['Runners On 2nd Before'] == 0) &
    (filtered_df['Runners On 3rd Before'] == 0)]

In [25]:
df_0o_11_12_03 = filtered_df[
    (filtered_df['Outs Before Play'] == 0) &
    (filtered_df['Runners On 1st Before'] == 1) &
    (filtered_df['Runners On 2nd Before'] == 1) &
    (filtered_df['Runners On 3rd Before'] == 0)]

In [27]:
df_1o_11_12_03 = filtered_df[
    (filtered_df['Outs Before Play'] == 1) &
    (filtered_df['Runners On 1st Before'] == 1) &
    (filtered_df['Runners On 2nd Before'] == 1) &
    (filtered_df['Runners On 3rd Before'] == 0)]

In [29]:
df_2o_11_12_03 = filtered_df[
    (filtered_df['Outs Before Play'] == 2) &
    (filtered_df['Runners On 1st Before'] == 1) &
    (filtered_df['Runners On 2nd Before'] == 1) &
    (filtered_df['Runners On 3rd Before'] == 0)]

In [31]:
df_0o_11_02_13 = filtered_df[
    (filtered_df['Outs Before Play'] == 0) &
    (filtered_df['Runners On 1st Before'] == 1) &
    (filtered_df['Runners On 2nd Before'] == 0) &
    (filtered_df['Runners On 3rd Before'] == 1)]

In [33]:
df_1o_11_02_13 = filtered_df[
    (filtered_df['Outs Before Play'] == 1) &
    (filtered_df['Runners On 1st Before'] == 1) &
    (filtered_df['Runners On 2nd Before'] == 0) &
    (filtered_df['Runners On 3rd Before'] == 1)]

In [35]:
df_2o_11_02_13 = filtered_df[
    (filtered_df['Outs Before Play'] == 2) &
    (filtered_df['Runners On 1st Before'] == 1) &
    (filtered_df['Runners On 2nd Before'] == 0) &
    (filtered_df['Runners On 3rd Before'] == 1)]

In [37]:
df_0o_11_12_13 = filtered_df[
    (filtered_df['Outs Before Play'] == 0) &
    (filtered_df['Runners On 1st Before'] == 1) &
    (filtered_df['Runners On 2nd Before'] == 1) &
    (filtered_df['Runners On 3rd Before'] == 1)]

In [39]:
df_1o_11_12_13 = filtered_df[
    (filtered_df['Outs Before Play'] == 1) &
    (filtered_df['Runners On 1st Before'] == 1) &
    (filtered_df['Runners On 2nd Before'] == 1) &
    (filtered_df['Runners On 3rd Before'] == 1)]

In [40]:
df_2o_11_12_13 = filtered_df[
    (filtered_df['Outs Before Play'] == 2) &
    (filtered_df['Runners On 1st Before'] == 1) &
    (filtered_df['Runners On 2nd Before'] == 1) &
    (filtered_df['Runners On 3rd Before'] == 1)]

In [43]:
df_0o_01_12_03 = filtered_df[
    (filtered_df['Outs Before Play'] == 0) &
    (filtered_df['Runners On 1st Before'] == 0) &
    (filtered_df['Runners On 2nd Before'] == 1) &
    (filtered_df['Runners On 3rd Before'] == 0)]

In [45]:
df_1o_01_12_03 = filtered_df[
    (filtered_df['Outs Before Play'] == 1) &
    (filtered_df['Runners On 1st Before'] == 0) &
    (filtered_df['Runners On 2nd Before'] == 1) &
    (filtered_df['Runners On 3rd Before'] == 0)]

In [46]:
df_2o_01_12_03 = filtered_df[
    (filtered_df['Outs Before Play'] == 2) &
    (filtered_df['Runners On 1st Before'] == 0) &
    (filtered_df['Runners On 2nd Before'] == 1) &
    (filtered_df['Runners On 3rd Before'] == 0)]

In [49]:
df_0o_01_12_13 = filtered_df[
    (filtered_df['Outs Before Play'] == 0) &
    (filtered_df['Runners On 1st Before'] == 0) &
    (filtered_df['Runners On 2nd Before'] == 1) &
    (filtered_df['Runners On 3rd Before'] == 1)]

In [50]:
df_1o_01_12_13 = filtered_df[
    (filtered_df['Outs Before Play'] == 1) &
    (filtered_df['Runners On 1st Before'] == 0) &
    (filtered_df['Runners On 2nd Before'] == 1) &
    (filtered_df['Runners On 3rd Before'] == 1)]

In [51]:
df_2o_01_12_13 = filtered_df[
    (filtered_df['Outs Before Play'] == 2) &
    (filtered_df['Runners On 1st Before'] == 0) &
    (filtered_df['Runners On 2nd Before'] == 1) &
    (filtered_df['Runners On 3rd Before'] == 1)]

In [55]:
df_0o_01_02_13 = filtered_df[
    (filtered_df['Outs Before Play'] == 0) &
    (filtered_df['Runners On 1st Before'] == 0) &
    (filtered_df['Runners On 2nd Before'] == 0) &
    (filtered_df['Runners On 3rd Before'] == 1)]

In [56]:
df_1o_01_02_13 = filtered_df[
    (filtered_df['Outs Before Play'] == 1) &
    (filtered_df['Runners On 1st Before'] == 0) &
    (filtered_df['Runners On 2nd Before'] == 0) &
    (filtered_df['Runners On 3rd Before'] == 1)]

In [57]:
df_2o_01_02_13 = filtered_df[
    (filtered_df['Outs Before Play'] == 2) &
    (filtered_df['Runners On 1st Before'] == 0) &
    (filtered_df['Runners On 2nd Before'] == 0) &
    (filtered_df['Runners On 3rd Before'] == 1)]

In [81]:
situational_dfs = [df_0o_01_02_03, 
                   df_1o_01_02_03, 
                   df_2o_01_02_03, 
                   df_0o_11_02_03, 
                   df_1o_11_02_03, 
                   df_2o_11_02_03,
                   df_0o_11_12_03, 
                   df_1o_11_12_03,
                   df_2o_11_12_03,
                   df_0o_11_02_13,
                   df_1o_11_02_13,
                   df_2o_11_02_13,
                   df_0o_11_12_13,
                   df_1o_11_12_13,
                   df_2o_11_12_13,
                   df_0o_01_12_03,
                   df_1o_01_12_03,
                   df_2o_01_12_03,
                   df_0o_01_12_13,
                   df_1o_01_12_13,
                   df_2o_01_12_13,
                   df_0o_01_02_13,
                   df_1o_01_02_13,
                   df_2o_01_02_13]

In [83]:
situations = ['0 Outs, No Runners On',
              '1 Outs, No Runners On',
              '2 Outs, No Runners On',
              '0 Outs, Runner on 1st',
              '1 Outs, Runner on 1st',
              '2 Outs, Runner on 1st',
              '0 Outs, Runner on 1st, Runner on 2nd',
              '1 Outs, Runner on 1st, Runner on 2nd',
              '2 Outs, Runner on 1st, Runner on 2nd',
              '0 Outs, Runner on 1st, Runner on 3rd',
              '1 Outs, Runner on 1st, Runner on 3rd',
              '2 Outs, Runner on 1st, Runner on 3rd',
              '0 Outs, Runner on 1st, Runner on 2nd, Runner on 3rd',
              '1 Outs, Runner on 1st, Runner on 2nd, Runner on 3rd',
              '2 Outs, Runner on 1st, Runner on 2nd, Runner on 3rd',
              '0 Outs, Runner on 2nd',
              '1 Outs, Runner on 2nd',
              '2 Outs, Runner on 2nd',
              '0 Outs, Runner on 2nd, Runner on 3rd',
              '1 Outs, Runner on 2nd, Runner on 3rd',
              '2 Outs, Runner on 2nd, Runner on 3rd',
              '0 Outs, Runner on 3rd',
              '1 Outs, Runner on 3rd',
              '2 Outs, Runner on 3rd',]

In [85]:
total_plays_situation = []
situation_runs_list = []
averages = []
for df in situational_dfs:
    if "Runs Scored Rest of Inning" in df.columns:
        total = df["Runs Scored Rest of Inning"].sum()
        situation_runs_list.append(total)

        total_plays = df.shape[0]
        total_plays_situation.append(total_plays)

count = 0
for item in total_plays_situation:
    average = situation_runs_list[count]/item
    averages.append(average)
    count += 1

In [86]:
final_situation_df = pd.DataFrame({
    'Situations': situations,
    'Total Plays': total_plays_situation,
    'Total Runs Scored': situation_runs_list,
    'Average Runs': averages
})
final_situation_df

,Situations,Total Plays,Total Runs Scored,Average Runs
0,"0 Outs, No Runners On",654,563,0.860856
1,"1 Outs, No Runners On",423,213,0.503546
2,"2 Outs, No Runners On",302,55,0.182119
3,"0 Outs, Runner on 1st",211,288,1.364929
4,"1 Outs, Runner on 1st",233,204,0.875536
5,"2 Outs, Runner on 1st",225,86,0.382222
6,"0 Outs, Runner on 1st, Runner on 2nd",76,130,1.710526
7,"1 Outs, Runner on 1st, Runner on 2nd",125,174,1.392000
8,"2 Outs, Runner on 1st, Runner on 2nd",123,68,0.552846
9,"0 Outs, Runner on 1st, Runner on 3rd",27,85,3.148148
